In [26]:
from google.colab import files
import zipfile
import os

# Upload ZIP file
uploaded = files.upload()  # Browse and select the ZIP file

# Extract the uploaded zip file (assuming it's named 'animal_data.zip')
with zipfile.ZipFile("animal_data.zip", 'r') as zip_ref:
    zip_ref.extractall("animal_data_file")  # Extracts to a folder in current directory

print("Extraction complete!")


Saving animal_data.zip to animal_data (2).zip
Extraction complete!


In [54]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
import os


**1. Problem Statement**

In [ ]:
# To detect animal name via image

**2. Data preprocessing (No augmentation)**

In [55]:
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

num_classes = train_generator.num_classes

Found 1561 images belonging to 15 classes.
Found 383 images belonging to 15 classes.


**3. Hypermodel using MobileNet**

In [56]:
def build_cnn_model(hp):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=image_size + (3,))
    base_model.trainable = False  # Freeze base

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())

    units = hp.Int("units", min_value=64, max_value=256, step=32)
    model.add(Dense(
        units=units,
        activation=hp.Choice("activation", ['relu', 'tanh']),
        kernel_initializer=hp.Choice("initializer", ['glorot_uniform', 'he_normal']),
        kernel_regularizer=regularizers.l2(
            hp.Float("lambda", 1e-4, 1e-2, sampling='log'))
    ))

    model.add(Dropout(hp.Float("dropout", 0.2, 0.5, step=0.1)))
    model.add(Dense(num_classes, activation='softmax'))

    learning_rate = hp.Float("learning_rate", 1e-4, 1e-2, sampling='log')
    optimizer_choice = hp.Choice("optimizer", ['adam', 'sgd', 'rmsprop'])

    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate)
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


**4. Keras Tuner Setup**

In [57]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='cnn_tuning',
    project_name='VGG16_model'
)

early_stop = EarlyStopping(monitor='val_loss', patience=5)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


**5. Start tuning**

In [58]:
tuner.search(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[early_stop],
    verbose=2
)

Trial 10 Complete [00h 02m 13s]
val_loss: 1.9813034534454346

Best val_loss So Far: 1.8218008279800415
Total elapsed time: 00h 20m 46s


**6. Save best model**

In [59]:
best_model = tuner.get_best_models(1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]

print("\nBest Hyperparameters:")
print(best_hyperparams.values)

if not os.path.exists("model"):
    os.mkdir("model")

best_model.save("model/VGG16_best_tuned_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



Best Hyperparameters:
{'units': 256, 'activation': 'relu', 'initializer': 'glorot_uniform', 'lambda': 0.0051112178288843175, 'dropout': 0.2, 'learning_rate': 0.0028298401894457986, 'optimizer': 'rmsprop'}


**Evaluation**

In [60]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Predict the class probabilities
y_pred_train_prob = best_model.predict(train_generator)

# Get the predicted class index (highest probability)
y_pred_train = np.argmax(y_pred_train_prob, axis=1)

# Get the true class labels
y_train = train_generator.classes

# Print evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(y_train, y_pred_train))

print("\nClassification Report:")
class_labels = list(train_generator.class_indices.keys())  # Get class names
print(classification_report(y_train, y_pred_train, target_names=class_labels))


49/49 ━━━━━━━━━━━━━━━━━━━━ 9s 174ms/step
Confusion Matrix:
[[32  4  5  3  5  6  9  5  5  3  5  2  1  8  7]
 [31  7  7  5  3  6 14  3  3  1  6  5  8  5  6]
 [33  5  4  2  2  5 16  7  6  3  3  3  1  6  3]
 [31  3  6  2  5  8  7  1 11  2  5  0  6 10  8]
 [29  7  4  4  3  4 11  7  8  2  4  6  3  6  4]
 [24  8  3  2  0  6 16  6  4  1  4  3  6  9  6]
 [37  5  7  3  3  1  7  6  4  4  5  3  5  5  9]
 [25  6  6  3  6  1 11  4  2  5  7  4  3 18  6]
 [32  9  3  3  3  7 17  4  3  2  3  3  2  5  8]
 [23  6  8  3  2 10 17  5  7  4  2  3  4  6  4]
 [27  5  7  5  5  2  7  9  2  2  1  6  9  9  5]
 [28  2  5  3  6  3 12  7  4  7  4  3  3 10  8]
 [32  7  9  6  1  7 10  3  2  2  4  6  4  6  9]
 [32  6  5  5  6  8 11  4  3  6  1  2  6  5  4]
 [26  5  4  4  5  6 15  3  1  9  6  1  5 11  9]]

Classification Report:
              precision    recall  f1-score   support

        Bear       0.07      0.32      0.12       100
        Bird       0.08      0.06      0.07       110
         Cat       0.05      0.04

In [61]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'animal_data_file/animal_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 1944 images belonging to 15 classes.


In [62]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np

# Step 1: Predict on test data
y_pred_test_prob = best_model.predict(test_generator)

# Step 2: Get the predicted class index (highest probability)
y_pred_test = np.argmax(y_pred_test_prob, axis=1)

# Step 3: Get the true class labels from the test generator
y_test = test_generator.classes

# Step 4: Class labels (same as training)
class_labels = list(test_generator.class_indices.keys())

# Step 5: Print metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, target_names=class_labels))


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 18s 301ms/step
Confusion Matrix:
[[115   0   1   1   0   2   5   1   0   0   0   0   0   0   0]
 [ 13  95   3   0   0   3  19   1   0   3   0   0   0   0   0]
 [ 40   0  73   0   0   3   5   0   0   0   0   1   1   0   0]
 [ 39   1   3  55   3   4  10   5   0   6   1   0   1   1   2]
 [ 30   1   2   2  56  10   7   0   3   5   4   0   0   6   1]
 [ 39   0   8   0   0  58   6   1   0   0   0   5   4   1   0]
 [ 14   0   0   0   0   0 115   0   0   0   0   0   0   0   0]
 [ 44   2   1   3   0   2   7  71   0   0   1   1   0   1   0]
 [ 32   2   0   1   3   1   5   1  79   0   4   0   0   1   0]
 [ 47   2   0   0   1   0  12   2   0  60   1   3   0   1   1]
 [ 37   3   4   0   1   4   8   4   3   2  56   0   0   4   0]
 [ 59   0   4   0   2   6   5   2   0   0   1  44   0   8   0]
 [ 32   0   5   0   0   9   9   0   0   0   0   2  77   1   0]
 [ 18   0   4   0   0   1   7   0   0   0   0   0   0  97   2]
 [  4   0   2   0   0   0   3   0   0   0   0   0   0  14 

In [63]:
print(train_generator.class_indices)


{'Bear': 0, 'Bird': 1, 'Cat': 2, 'Cow': 3, 'Deer': 4, 'Dog': 5, 'Dolphin': 6, 'Elephant': 7, 'Giraffe': 8, 'Horse': 9, 'Kangaroo': 10, 'Lion': 11, 'Panda': 12, 'Tiger': 13, 'Zebra': 14}


In [64]:
train_generator = datagen.flow_from_directory(
    "animal_data_file/animal_data",          # NOT the folder with all images directly inside
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)



Found 1561 images belonging to 15 classes.


In [65]:
val_prob = best_model.predict(val_generator)
val_pred = np.argmax(val_prob, axis=1)
val_true = val_generator.classes

from sklearn.metrics import classification_report
print(classification_report(val_true, val_pred, target_names=class_labels))


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step
              precision    recall  f1-score   support

        Bear       0.19      0.92      0.32        25
        Bird       0.71      0.56      0.62        27
         Cat       0.37      0.42      0.39        24
         Cow       0.56      0.19      0.29        26
        Deer       0.91      0.40      0.56        25
         Dog       0.35      0.33      0.34        24
     Dolphin       0.49      0.84      0.62        25
    Elephant       0.79      0.42      0.55        26
     Giraffe       1.00      0.80      0.89        25
       Horse       0.43      0.38      0.41        26
    Kangaroo       0.75      0.24      0.36        25
        Lion       0.67      0.15      0.25        26
       Panda       0.88      0.56      0.68        27
       Tiger       0.69      0.44      0.54        25
       Zebra       0.96      0.85      0.90        27

    accuracy                           0.50       383
   macro avg       0.65      0.50      

### Evaluate Accuracy on Validation/Test Data

In [66]:
val_loss, val_accuracy = best_model.evaluate(val_generator, verbose=1)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Validation Loss: {val_loss:.4f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 219ms/step - accuracy: 0.5441 - loss: 1.7450
Validation Accuracy: 50.13%
Validation Loss: 1.8218
